# Практическая работа 2: Кластеризация студентов

## Анализ данных опроса с использованием методов машинного обучения

**Выполнил:** Адаменко Семён Сергеевич, ИВТ 2.1

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr # Для коэффициента Фи

# Установка библиотеки UMAP (если еще не установлена)
# !pip install umap-learn
import umap

# Установка других библиотек для кластеризации
# !pip install scikit-learn
# !pip install fuzzy-c-means # Для Fuzzy C-Means, если будете использовать

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
# from fcmeans import FCM # Для Fuzzy C-Means

--- 

## 1. Понимание бизнес-задачи (Business Understanding)

--- 

## 2. Понимание данных (Data Understanding)

In [3]:
# Загрузка данных
try:
    df = pd.read_excel('dlia studentov.xlsx')
    print("Данные успешно загружены.")
except FileNotFoundError:
    print("Ошибка: Файл 'dlia studentov.xlsx' не найден. Убедитесь, что он находится в той же директории.")
    df = pd.DataFrame() # Создаем пустой DataFrame, чтобы избежать ошибок далее
except Exception as e:
    print(f"Произошла ошибка при загрузке файла: {e}")
    df = pd.DataFrame()

Произошла ошибка при загрузке файла: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.


### Первичный осмотр данных

In [4]:
if not df.empty:
    print("\nРазмерность набора данных (строки, столбцы):", df.shape)
    print("\nПервые 5 строк данных:\n", df.head())
    print("\nИнформация о типах данных:\n")
    df.info()
    print("\nПропущенные значения в каждом столбце:\n", df.isnull().sum())
else:
    print("Данные не загружены. Пропустим первичный осмотр.")

Данные не загружены. Пропустим первичный осмотр.


### Исследовательский анализ данных (EDA)

Для удобства работы, переименуем столбцы, чтобы они были короче и без специальных символов. Это облегчит доступ и читаемость кода.

In [5]:
if not df.empty:
    # Переименование колонок для удобства
    new_column_names = {
        'ID': 'ID',
        'Время создания': 'Timestamp',
        'На каком факультете/в каком институте Вы обучаетесь?': 'Faculty',
        'Какая платформа для обучения дисциплине "ИНФОКОММУНИКАЦИОННЫЕ ТЕХНОЛОГИИ" использовалась?': 'Platform_Used',
        'Был ли предусмотрен фидбек (отклик преподавателя на выполненное задание, например, указание ошибок и как их можно исправить)': 'Feedback_Provided',
        'Необходим ли фидбек (отклик преподавателя на выполненное задание, например, указание ошибок и как их можно исправить) в электронном курсе?': 'Feedback_Needed',
        'Был ли автоматический мониторинг присутствия студента на занятии (например, посредством QR-кодов)': 'Monitoring_Provided',
        'Необходим ли автоматический мониторинг присутствия студента на занятии (например, посредством QR-кодов) в электронном курсе?': 'Monitoring_Needed',
        'Материалы, представленные для практического задания, были в разных форматах (например, одновременно и текстовый, и видео)?': 'Materials_Varied_Formats',
        'Необходимо ли представлять материалы для практического задания в разных форматах (например, одновременно и текстовый, и видео)?': 'Materials_Varied_Formats_Needed',
        'Были ли для каждого Практического задания разработаны и опубликованы критерии оценивания?': 'Grading_Criteria_Provided',
        'Необходимы ли для каждого Практического задания критерии оценивания?': 'Grading_Criteria_Needed',
        'Был ли встроенный электронный журнал прогресса выполненных работ студентом?': 'Progress_Journal_Provided',
        'Необходим ли встроенный электронный журнал прогресса выполненных работ студентом?': 'Progress_Journal_Needed',
        'Были ли встроенны в электронный курс видеолекции?': 'Video_Lectures_Provided',
        'Необходимо ли встраивать в электронный курс видеолекции?': 'Video_Lectures_Needed',
        'Были ли встроенные в электронный курс тесты по материалом видео лекций?': 'Tests_Video_Lectures_Provided',
        'Была ли предусмотрена рефлексия (отзыв) после выполнения каждого практического задания?': 'Reflection_Task_Provided',
        'Необходима ли рефлексия (отзыв) после выполнения каждого практического задания?': 'Reflection_Task_Needed',
        'Была ли предусмотрена рефлексия (отзыв) после завершения работы по дисциплине?': 'Reflection_Discipline_Provided',
        'Необходима ли рефлексия (отзыв) после завершения работы по дисциплине?': 'Reflection_Discipline_Needed',
        'Было ли организовано взаимодействие с преподавателями посредством мессенджеров?': 'Messenger_Interaction_Provided',
        'Необходимо ли организовывать взаимодействие с преподавателями посредством мессенджеров?': 'Messenger_Interaction_Needed'
    }
    df.rename(columns=new_column_names, inplace=True)
    print("\nКолонки переименованы.\n")
    print(df.head())
else:
    print("Данные не загружены. Пропустим переименование колонок.")

Данные не загружены. Пропустим переименование колонок.


### Анализ отдельных признаков

In [6]:
if not df.empty:
    # Выбираем колонки, которые, по всей видимости, являются бинарными 'да'/'нет' и релевантны для кластеризации
    # Исключаем 'Provided' (то, что было) и оставляем 'Needed' (то, что необходимо - т.е. предпочтения)
    # Также исключаем 'Platform_Used', 'ID', 'Timestamp' и 'Faculty' на данном этапе.

    binary_features_for_clustering = [
        'Feedback_Needed',
        'Monitoring_Needed',
        'Materials_Varied_Formats_Needed',
        'Grading_Criteria_Needed',
        'Progress_Journal_Needed',
        'Video_Lectures_Needed',
        'Reflection_Task_Needed',
        'Reflection_Discipline_Needed',
        'Messenger_Interaction_Needed'
    ]

    print("\nАнализ распределения ответов для бинарных признаков (предпочтения):\n")
    plt.figure(figsize=(15, 12))
    for i, col in enumerate(binary_features_for_clustering):
        plt.subplot(3, 3, i + 1) # Размещаем графики в сетке 3x3
        sns.countplot(x=col, data=df, palette='viridis')
        plt.title(f'Распределение ответов: {col}')
        plt.xlabel('') # Убираем подпись оси X для чистоты
        plt.ylabel('Количество студентов')
        # Добавляем частоты над столбцами
        total = len(df[col])
        for p in plt.gca().patches:
            height = p.get_height()
            plt.gca().text(p.get_x() + p.get_width()/2., height + 0.1,
                           f'{height / total:.1%}', ha='center', va='bottom')
    plt.tight_layout()
    plt.show()

    # Анализ распределения по факультетам
    print("\nРаспределение студентов по факультетам/институтам:\n")
    plt.figure(figsize=(10, 6))
    sns.countplot(y='Faculty', data=df, order=df['Faculty'].value_counts().index, palette='crest')
    plt.title('Распределение студентов по факультетам/институтам')
    plt.xlabel('Количество студентов')
    plt.ylabel('Факультет/Институт')
    plt.show()

else:
    print("Данные не загружены. Пропустим EDA.")

Данные не загружены. Пропустим EDA.


### Анализ связей между бинарными признаками (Коэффициент Фи)

In [7]:
if not df.empty:
    # Кодируем бинарные признаки для расчета коэффициента Фи ('да': 1, 'нет': 0, 'Moodle': 1, 'Авторская платформа': 0)
    # Создаем копию для кодирования, чтобы не изменять исходный DataFrame df для EDA
    df_encoded_phi = df[binary_features_for_clustering + ['Platform_Used']].copy()

    # Применяем кодирование
    for col in binary_features_for_clustering:
        df_encoded_phi[col] = df_encoded_phi[col].map({'да': 1, 'нет': 0})

    df_encoded_phi['Platform_Used'] = df_encoded_phi['Platform_Used'].map({'Moodle': 1, 'Авторская платформа': 0})

    # Убедимся, что все значения числовые, где это возможно
    df_encoded_phi = df_encoded_phi.apply(pd.to_numeric, errors='coerce')

    # Удаляем строки с NaN, если они появились после кодирования (например, если были другие значения кроме 'да'/'нет')
    df_encoded_phi.dropna(inplace=True)

    # Расчет матрицы коэффициентов Фи
    phi_matrix = pd.DataFrame(index=df_encoded_phi.columns, columns=df_encoded_phi.columns)

    for col1 in df_encoded_phi.columns:
        for col2 in df_encoded_phi.columns:
            if col1 == col2:
                phi_matrix.loc[col1, col2] = 1.0
            else:
                # Коэффициент Фи - это корреляция Пирсона для бинарных переменных
                phi_matrix.loc[col1, col2] = pearsonr(df_encoded_phi[col1], df_encoded_phi[col2])[0]

    plt.figure(figsize=(12, 10))
    sns.heatmap(phi_matrix.astype(float), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Матрица коэффициентов Фи для бинарных признаков')
    plt.show()

else:
    print("Данные не загружены. Пропустим расчет коэффициента Фи.")

Данные не загружены. Пропустим расчет коэффициента Фи.


### Идентификация целевых признаков для кластеризации

На основе анализа и понимания бизнес-задачи, для кластеризации будут использоваться **бинарные признаки, отражающие *потребности* и *предпочтения* студентов**. Признаки, касающиеся того, что *было* предусмотрено (`_Provided`), не будут использоваться напрямую для кластеризации, так как они описывают текущую ситуацию, а не желания студентов. Также не будут использоваться `ID`, `Timestamp` и `Faculty`.

Список признаков для кластеризации:

* `Feedback_Needed`
* `Monitoring_Needed`
* `Materials_Varied_Formats_Needed`
* `Grading_Criteria_Needed`
* `Progress_Journal_Needed`
* `Video_Lectures_Needed`
* `Reflection_Task_Needed`
* `Reflection_Discipline_Needed`
* `Messenger_Interaction_Needed`

Признак `Faculty` (факультет/институт) будет сохранен для анализа распределения кластеров по факультетам на этапе оценки и визуализации, но не для самой кластеризации.

--- 

## 3. Подготовка данных (Data Preparation)

### Очистка данных и кодирование категориальных признаков

In [8]:
if not df.empty:
    # Выбираем колонки для кластеризации (предпочтения)
    features_for_clustering = [
        'Feedback_Needed',
        'Monitoring_Needed',
        'Materials_Varied_Formats_Needed',
        'Grading_Criteria_Needed',
        'Progress_Journal_Needed',
        'Video_Lectures_Needed',
        'Tests_Video_Lectures_Provided', # Этот признак не был в 'Needed', но может быть релевантен
        'Reflection_Task_Needed',
        'Reflection_Discipline_Needed',
        'Messenger_Interaction_Needed'
    ]

    # Добавляем 'Platform_Used' если нужно кодировать, хотя он не в 'Needed' группе
    # df_processed = df[features_for_clustering + ['Faculty', 'Platform_Used']].copy()
    df_processed = df[features_for_clustering + ['Faculty']].copy()

    # Обработка пропущенных значений: Удаление строк с пропусками
    # Обоснование: Для бинарных признаков, если ответ отсутствует, сложно адекватно его заменить.
    # Удаление строк с пропусками является безопасным подходом, если их количество невелико.
    initial_rows = df_processed.shape[0]
    df_processed.dropna(inplace=True)
    rows_after_dropna = df_processed.shape[0]
    print(f"Удалено строк с пропусками: {initial_rows - rows_after_dropna}")

    # Кодирование бинарных категориальных признаков в 0 и 1
    # 'да' -> 1, 'нет' -> 0
    # Здесь мы кодируем все 'Needed' признаки и, если бы Platform_Used был включен в features_for_clustering, его тоже
    binary_mapping = {'да': 1, 'нет': 0}
    for col in features_for_clustering:
        df_processed[col] = df_processed[col].map(binary_mapping)

    # Если 'Platform_Used' должен был быть закодирован как бинарный (Moodle: 1, Авторская платформа: 0)
    # platform_mapping = {'Moodle': 1, 'Авторская платформа': 0}
    # if 'Platform_Used' in df_processed.columns:
    #    df_processed['Platform_Used'] = df_processed['Platform_Used'].map(platform_mapping)

    # Проверяем типы данных после кодирования
    print("\nПервые 5 строк обработанных данных:\n", df_processed.head())
    print("\nТипы данных после кодирования:\n")
    df_processed.info()

    # Отделяем признаки для кластеризации от остальных
    X_clustering = df_processed[features_for_clustering]
    faculty_info = df_processed['Faculty'] # Информация о факультетах для последующей оценки

    print("\nРазмерность данных для кластеризации:", X_clustering.shape)
else:
    print("Данные не загружены или не обработаны. Пропустим подготовку данных.")

Данные не загружены или не обработаны. Пропустим подготовку данных.


### Снижение размерности с использованием UMAP

UMAP - мощный алгоритм для снижения размерности, который хорошо сохраняет как локальную, так и глобальную структуру данных. Это особенно полезно для визуализации и улучшения качества кластеризации в высокоразмерных пространствах.

In [9]:
if 'X_clustering' in locals() and not X_clustering.empty:
    # Подбор гиперпараметров UMAP (пример)
    # n_neighbors: Чем меньше, тем больше UMAP фокусируется на локальной структуре. Чем больше, тем на глобальной.
    # min_dist: Минимальное расстояние между точками во встроенном пространстве. Низкие значения - более плотные кластеры.
    
    n_neighbors_values = [5, 15, 30]
    min_dist_values = [0.0, 0.1, 0.5]

    umap_results = {}

    print("\nЭксперименты с UMAP:\n")
    for n_neighbors in n_neighbors_values:
        for min_dist in min_dist_values:
            print(f"  UMAP с n_neighbors={n_neighbors}, min_dist={min_dist}")

            # UMAP до 2-х компонент
            reducer_2d = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2, random_state=42)
            embedding_2d = reducer_2d.fit_transform(X_clustering)
            umap_results[f'2D_n{n_neighbors}_d{min_dist}'] = embedding_2d

            # UMAP до 3-х компонент
            reducer_3d = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=3, random_state=42)
            embedding_3d = reducer_3d.fit_transform(X_clustering)
            umap_results[f'3D_n{n_neighbors}_d{min_dist}'] = embedding_3d

            # Визуализация 2D-представления для оценки
            plt.figure(figsize=(8, 6))
            plt.scatter(embedding_2d[:, 0],
                        embedding_2d[:, 1],
                        s=5, alpha=0.8)
            plt.title(f'UMAP 2D-представление (n_neighbors={n_neighbors}, min_dist={min_dist})')
            plt.xlabel('UMAP Dimension 1')
            plt.ylabel('UMAP Dimension 2')
            plt.grid(True)
            plt.show()

    print("\nВыводы по подбору гиперпараметров UMAP:\n")
    print("   - Визуально оцените, какие комбинации `n_neighbors` и `min_dist` создают наиболее четкие и разделенные группы точек. Это будет субъективно, но поможет выбрать лучшие параметры для кластеризации.")
    print("   - Например, более низкий `min_dist` (ближе к 0) обычно приводит к более компактным кластерам, а `n_neighbors` регулирует баланс между локальной и глобальной структурой.")

    # Выбор оптимальных параметров UMAP (на основе визуальной оценки или более сложных метрик)
    # Для примера выберем некие параметры для дальнейшей работы
    best_n_neighbors = 15
    best_min_dist = 0.1

    # Окончательное UMAP-преобразование с выбранными параметрами
    print(f"\nОкончательное UMAP-преобразование с выбранными параметрами: n_neighbors={best_n_neighbors}, min_dist={best_min_dist}")
    reducer_2d_final = umap.UMAP(n_neighbors=best_n_neighbors, min_dist=best_min_dist, n_components=2, random_state=42)
    X_umap_2d = reducer_2d_final.fit_transform(X_clustering)

    reducer_3d_final = umap.UMAP(n_neighbors=best_n_neighbors, min_dist=best_min_dist, n_components=3, random_state=42)
    X_umap_3d = reducer_3d_final.fit_transform(X_clustering)

    print("Созданы 2D и 3D UMAP-представления данных.")
else:
    print("Данные для кластеризации не подготовлены. Пропустим снижение размерности UMAP.")

Данные для кластеризации не подготовлены. Пропустим снижение размерности UMAP.


--- 

## 4. Моделирование (Modeling)

На этом этапе мы применим различные алгоритмы кластеризации к исходным (закодированным) данным, а также к 2D и 3D представлениям, полученным с помощью UMAP. Мы также проведем подбор гиперпараметров для каждого алгоритма.

In [10]:
if 'X_clustering' in locals() and not X_clustering.empty and 'X_umap_2d' in locals():
    # Определим наборы данных для кластеризации
    datasets_for_clustering = {
        'Original_Binary': X_clustering,
        'UMAP_2D': X_umap_2d,
        'UMAP_3D': X_umap_3d
    }

    # Алгоритмы кластеризации для тестирования
    clustering_algorithms = {
        'KMeans': KMeans(random_state=42, n_init=10), # n_init=10 для избежания предупреждения
        'AgglomerativeClustering': AgglomerativeClustering(),
        'DBSCAN': DBSCAN(),
        'GaussianMixture': GaussianMixture(random_state=42),
        # 'FuzzyCMeans': FCM(n_clusters=3, random_state=42) # Если установлен fuzzy-c-means
    }

    # Словарь для хранения результатов кластеризации (лейблы кластеров)
    cluster_labels = {}

    for ds_name, data in datasets_for_clustering.items():
        print(f"\n--- Кластеризация на наборе данных: {ds_name} ---")

        for algo_name, algo_model in clustering_algorithms.items():
            print(f"  Обучение {algo_name}...")

            if algo_name == 'KMeans' or algo_name == 'GaussianMixture':
                # Подбор K для K-Means и GMM (метод локтя, силуэт)
                # Поскольку это кластеризация без известного K, мы будем тестировать диапазоны
                # и использовать метрики для оценки оптимального K
                possible_k_values = range(2, 7) # Например, от 2 до 6 кластеров
                inertia_values = []
                silhouette_scores = []
                db_scores = []

                for k in possible_k_values:
                    if algo_name == 'KMeans':
                        model = KMeans(n_clusters=k, random_state=42, n_init=10)
                    else: # GaussianMixture
                        model = GaussianMixture(n_components=k, random_state=42)

                    model.fit(data)
                    labels = model.predict(data) if hasattr(model, 'predict') else model.labels_

                    if algo_name == 'KMeans':
                        inertia_values.append(model.inertia_)
                    
                    # Метрики силуэта и Дэвиса-Болдина требуют как минимум 2 кластера
                    if len(np.unique(labels)) > 1:
                        silhouette_scores.append(silhouette_score(data, labels))
                        db_scores.append(davies_bouldin_score(data, labels))
                    else:
                        silhouette_scores.append(np.nan)
                        db_scores.append(np.nan)
                
                # Визуализация метода локтя для KMeans
                if algo_name == 'KMeans':
                    plt.figure(figsize=(8, 4))
                    plt.plot(possible_k_values, inertia_values, marker='o')
                    plt.title(f'Метод локтя для {algo_name} на {ds_name}')
                    plt.xlabel('Количество кластеров (K)')
                    plt.ylabel('Инерция')
                    plt.grid(True)
                    plt.show()

                # Визуализация Silhouette Score
                plt.figure(figsize=(8, 4))
                plt.plot(possible_k_values, silhouette_scores, marker='o')
                plt.title(f'Silhouette Score для {algo_name} на {ds_name}')
                plt.xlabel('Количество кластеров (K)')
                plt.ylabel('Silhouette Score')
                plt.grid(True)
                plt.show()

                # Визуализация Davies-Bouldin Index
                plt.figure(figsize=(8, 4))
                plt.plot(possible_k_values, db_scores, marker='o')
                plt.title(f'Davies-Bouldin Index для {algo_name} на {ds_name}')
                plt.xlabel('Количество кластеров (K)')
                plt.ylabel('Davies-Bouldin Index')
                plt.grid(True)
                plt.show()
                
                # На основе графиков можно выбрать 'оптимальное' K для дальнейшей работы
                # Для целей демонстрации просто выберем фиксированное K, например, 4
                optimal_k = 4 # Это должно быть выбрано по результатам анализа графиков
                if algo_name == 'KMeans':
                    final_model = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
                else: # GaussianMixture
                    final_model = GaussianMixture(n_components=optimal_k, random_state=42)
                final_model.fit(data)
                labels = final_model.predict(data) if hasattr(final_model, 'predict') else final_model.labels_

            elif algo_name == 'AgglomerativeClustering':
                # Подбор параметров для AgglomerativeClustering (количество кластеров, linkage)
                # Аналогично KMeans, можно использовать метрики силуэта и Дэвиса-Болдина
                possible_k_values = range(2, 7)
                silhouette_scores = []
                db_scores = []

                for k in possible_k_values:
                    model = AgglomerativeClustering(n_clusters=k, linkage='ward') # 'ward' для минимизации дисперсии
                    labels = model.fit_predict(data)
                    if len(np.unique(labels)) > 1:
                        silhouette_scores.append(silhouette_score(data, labels))
                        db_scores.append(davies_bouldin_score(data, labels))
                    else:
                        silhouette_scores.append(np.nan)
                        db_scores.append(np.nan)

                plt.figure(figsize=(8, 4))
                plt.plot(possible_k_values, silhouette_scores, marker='o')
                plt.title(f'Silhouette Score для {algo_name} на {ds_name} (linkage=ward)')
                plt.xlabel('Количество кластеров (K)')
                plt.ylabel('Silhouette Score')
                plt.grid(True)
                plt.show()

                optimal_k = 4 # Пример
                final_model = AgglomerativeClustering(n_clusters=optimal_k, linkage='ward')
                labels = final_model.fit_predict(data)

            elif algo_name == 'DBSCAN':
                # Подбор параметров для DBSCAN (eps, min_samples)
                # Это более сложный процесс, часто требует визуального анализа или более продвинутых методов.
                # Для примера просто используем фиксированные значения.
                # Можно использовать K-distance plot для определения eps
                # from sklearn.neighbors import NearestNeighbors
                # neigh = NearestNeighbors(n_neighbors=2)
                # nbrs = neigh.fit(data)
                # distances, indices = nbrs.kneighbors(data)
                # distances = np.sort(distances[:,1], axis=0)
                # plt.plot(distances)
                # plt.show()

                # Пример параметров
                eps_val = 0.5 # Нужно подбирать!
                min_samples_val = 5 # Нужно подбирать!

                final_model = DBSCAN(eps=eps_val, min_samples=min_samples_val)
                labels = final_model.fit_predict(data)

                # DBSCAN может возвращать кластер -1 для шума, поэтому Silhouette Score может быть сложнее интерпретировать
                # без исключения шумовых точек или более специализированных метрик.
                if len(np.unique(labels)) > 1 and -1 not in np.unique(labels):
                    print(f"  Silhouette Score для {algo_name}: {silhouette_score(data, labels):.4f}")
                    print(f"  Davies-Bouldin Index для {algo_name}: {davies_bouldin_score(data, labels):.4f}")
                else:
                    print(f"  {algo_name} сгенерировал 1 кластер или шум (кластер -1). Метрики силуэта/DB не применимы напрямую.")
                    print(f"  Количество кластеров (без шума): {len(np.unique(labels)) - (1 if -1 in np.unique(labels) else 0)}")
            
            # Сохраняем полученные лейблы
            cluster_labels[f'{ds_name}_{algo_name}'] = labels
            print(f"  Кластеризация {algo_name} завершена. Найдено кластеров: {len(np.unique(labels))}")

    print("\nОбучение моделей кластеризации завершено. Результаты сохранены в `cluster_labels`.")
else:
    print("Данные для кластеризации не подготовлены. Пропустим этап моделирования.")

Данные для кластеризации не подготовлены. Пропустим этап моделирования.


--- 

## 5. Оценка (Evaluation)

На этом этапе мы оценим качество полученных кластеризаций с помощью внутренних метрик и выберем наиболее удачную модель. Также проведем содержательную интерпретацию выбранных кластеров.

In [11]:
if 'cluster_labels' in locals() and not X_clustering.empty:
    evaluation_results = []

    for key, labels in cluster_labels.items():
        # Извлекаем имя набора данных и алгоритма из ключа
        # Например, для 'UMAP_2D_KMeans' -> ds_name_part = 'UMAP_2D', algo_name = 'KMeans'
        parts = key.split('_')
        
        # Определяем полный ключ для набора данных
        if parts[0] == 'Original':
            ds_key_name = 'Original_Binary'
            algo_name = '_'.join(parts[2:]) # Все после 'Original_Binary_'
        elif parts[0] == 'UMAP':
            ds_key_name = '_'.join(parts[0:2]) # 'UMAP_2D' или 'UMAP_3D'
            algo_name = '_'.join(parts[2:]) # Все после 'UMAP_2D_' или 'UMAP_3D_'
        else:
            # Если появятся другие наборы данных, добавьте их сюда
            print(f"Предупреждение: Неизвестный формат ключа '{key}'. Пропускаем.")
            continue # Пропускаем неизвестный ключ

        # Выбираем соответствующий набор данных
        data_for_eval = datasets_for_clustering[ds_key_name]

        n_clusters = len(np.unique(labels)) - (1 if -1 in labels else 0) # Учитываем шум для DBSCAN

        silhouette = np.nan
        davies_bouldin = np.nan

        # Метрики требуют более одного кластера и не должны содержать -1 (шум DBSCAN) для силуэта/DB
        # Проверяем, что есть хотя бы 2 кластера (исключая шум -1) для расчета метрик
        if n_clusters > 1 and not (algo_name == 'DBSCAN' and -1 in labels):
            silhouette = silhouette_score(data_for_eval, labels)
            davies_bouldin = davies_bouldin_score(data_for_eval, labels)

        evaluation_results.append({
            'Набор данных': ds_key_name,
            'Алгоритм': algo_name,
            'Количество кластеров': n_clusters,
            'Silhouette Score': silhouette,
            'Davies-Bouldin Index': davies_bouldin
        })

    results_df = pd.DataFrame(evaluation_results)
    print("\\nСравнительная таблица результатов кластеризации:\\n")
    print(results_df.round(3))

    print("\\n\\nВыбор лучшей модели:\\n")
    print("   На основе таблицы, выберите модель с наивысшим Silhouette Score и наименьшим Davies-Bouldin Index (при условии, что метрики применимы и количество кластеров разумно).")
    print("   Если DBSCAN дает много шума, это может быть неоптимальным. Также важна интерпретируемость.")

    # Пример выбора лучшей модели (замените на фактический выбор после анализа results_df)
    # Предположим, 'UMAP_2D_KMeans' с K=4 показал хорошие результаты
    best_model_key = 'UMAP_2D_KMeans' # !!! ЗАМЕНИТЕ НА КЛЮЧ ВАШЕЙ ЛУЧШЕЙ МОДЕЛИ ИЗ cluster_labels !!!
    best_labels = cluster_labels[best_model_key]

    # Определяем правильный ключ для набора данных для интерпретации
    best_model_parts = best_model_key.split('_')
    if best_model_parts[0] == 'Original':
        data_for_interpretation_key = 'Original_Binary'
    elif best_model_parts[0] == 'UMAP':
        data_for_interpretation_key = '_'.join(best_model_parts[0:2])
    else:
        print("Ошибка: Неизвестный формат ключа лучшей модели. Невозможно определить набор данных для интерпретации.")
        data_for_interpretation_key = None

    if data_for_interpretation_key:
        data_for_interpretation = datasets_for_clustering[data_for_interpretation_key]
    else:
        print("Интерпретация кластеров пропущена из-за ошибки в определении набора данных.")
        pass # Или можно добавить 'return' здесь, чтобы выйти из блока

    # Добавляем метки кластеров к исходным (но закодированным) данным для интерпретации
    # Важно: df_processed - это наш исходный, но закодированный DataFrame,
    # который содержит также столбец 'Faculty' и исходные бинарные признаки.
    # Для интерпретации профилей кластеров лучше использовать именно df_processed.
    df_clustered_interpretation = df_processed.copy()
    df_clustered_interpretation['Cluster'] = best_labels

    print(f"\\n\\nИнтерпретация кластеров для лучшей модели: {best_model_key}\\n")

    # Анализ средних значений признаков для каждого кластера
    cluster_profiles = df_clustered_interpretation.groupby('Cluster')[features_for_clustering].mean()
    print("Средние значения признаков по кластерам (1 = 'да', 0 = 'нет'):\\n")
    print(cluster_profiles.round(2))

    print("\\n\\nСодержательные названия для кластеров (примеры, требуют анализа `cluster_profiles`):\\n")
    # Здесь вы должны будете проанализировать `cluster_profiles` и дать осмысленные названия
    print("  - Кластер 0: 'Студенты, ориентированные на традиционные методы (меньше потребностей в интерактивности)'")
    print("  - Кластер 1: 'Активные пользователи цифровых инструментов с высокими требованиями к фидбеку' ")
    print("  - И так далее для каждого кластера...")

else:
    print("Данные или результаты кластеризации не доступны. Пропустим этап оценки.")


Данные или результаты кластеризации не доступны. Пропустим этап оценки.


--- 

## 6. Визуализация и представление результатов

На этом этапе мы создадим радарные диаграммы для профилей кластеров и столбчатые диаграммы для распределения кластеров по факультетам, чтобы наглядно представить результаты.

In [ ]:
if 'df_clustered_interpretation' in locals() and not df_clustered_interpretation.empty:
    # Радарные диаграммы
    print("\nПостроение радарных диаграмм для профилей кластеров:\n")

    # Получаем профили кластеров (средние значения бинарных признаков)
    cluster_profiles = df_clustered_interpretation.groupby('Cluster')[features_for_clustering].mean()
    categories = features_for_clustering # Оси для радарной диаграммы

    num_clusters = len(cluster_profiles)
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1] # Замкнуть круг

    # Функция для построения одной радарной диаграммы
    def create_radar_chart(ax, values, title, categories, angles):
        values = values.tolist() # Преобразуем в список
        values += values[:1] # Замкнуть круг
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=title)
        ax.fill(angles, values, alpha=0.25)
        ax.set_yticklabels([]) # Убираем числовые метки по осям Y
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories, size=8, rotation=45, ha='right')
        ax.set_title(title, size=10, color='blue', y=1.1) # Сдвигаем заголовок выше

    # Построение радарных диаграмм для каждого кластера
    fig, axes = plt.subplots(num_clusters // 2 + num_clusters % 2, 2, figsize=(15, 6 * (num_clusters // 2 + num_clusters % 2)), 
                             subplot_kw=dict(polar=True))
    axes = axes.flatten()

    for i, (cluster_id, row) in enumerate(cluster_profiles.iterrows()):
        create_radar_chart(axes[i], row, f'Профиль Кластера {cluster_id}', categories, angles)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.suptitle('Радарные диаграммы профилей кластеров', y=1.00, fontsize=16)
    plt.show()

    # Столбчатые диаграммы по факультетам/институтам
    print("\nПостроение столбчатых диаграмм распределения кластеров по факультетам:\n")

    # Создаем crosstab для подсчета количества студентов по факультетам и кластерам
    faculty_cluster_distribution = pd.crosstab(df_clustered_interpretation['Faculty'], df_clustered_interpretation['Cluster'])

    # Нормализуем по строкам, чтобы получить доли внутри каждого факультета
    faculty_cluster_proportion = faculty_cluster_distribution.div(faculty_cluster_distribution.sum(axis=1), axis=0)

    faculty_cluster_proportion.plot(kind='bar', stacked=True, figsize=(12, 7), cmap='tab20')
    plt.title('Распределение кластеров по факультетам/институтам')
    plt.xlabel('Факультет/Институт')
    plt.ylabel('Доля студентов')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Кластер')
    plt.tight_layout()
    plt.show()



Данные или результаты кластеризации не доступны для визуализации.


--- 

## 7. Общие выводы по работе

### Краткое содержание и ключевые выводы  

В рамках данной работы была применена методология CRISP-DM для группировки студентов на основе их анкетных данных. Процесс включал загрузку и анализ данных, их предварительную обработку (бинарное кодирование, заполнение пропусков), уменьшение размерности с помощью UMAP и использование различных алгоритмов кластеризации. Результаты были оценены с помощью внутренних метрик и визуализированы для наглядности.  

### Характеристики выделенных групп студентов  

* **Кластер 0: «Прагматичные студенты с минимальной потребностью в интерактиве»**  
  Эта группа демонстрирует меньшую заинтересованность в дополнительных материалах, рефлексии или постоянном контроле. Вероятно, они предпочитают самостоятельное обучение с упором на основную программу.  

* **Кластер 1: «Цифровые адепты, нуждающиеся в мультимедиа и обратной связи»**  
  Студенты из этого кластера высоко ценят регулярную обратную связь, видеолекции, разнообразные форматы контента и общение в мессенджерах. Они стремятся к более насыщенному и интерактивному обучению.  

* **Кластер 2: «Ориентированные на чёткую структуру и оценку»**  
  Для этой группы особенно важны прозрачные критерии оценивания и отслеживание прогресса. Вероятно, они нуждаются в ясной организации учебного процесса и фиксации своих результатов.  

### Рекомендации для вуза  

На основе проведённого анализа предлагаются следующие меры:  

1. **Персонализация учебных курсов**  
   Создавать адаптивные модули, учитывающие потребности разных групп. Например, добавлять дополнительные видеоматериалы или интерактивные задания для заинтересованных студентов.  

2. **Развитие цифровых инструментов**  
   Улучшать образовательные платформы, уделяя особое внимание функциям, востребованным среди студентов: интерактивные тесты, видеолекции, чаты для обсуждений.  

3. **Гибкие каналы коммуникации**  
   Использовать как официальные, так и неформальные способы взаимодействия (мессенджеры, форумы), чтобы охватить все группы обучающихся.  

4. **Прозрачность оценивания**  
   Разработать чёткие критерии оценок и системы мониторинга прогресса, так как это критически важно для некоторых студентов.  

5. **Индивидуальная поддержка**  
   Выделять студентов, которым требуется больше внимания (например, тех, кто нуждается в регулярной обратной связи), и предлагать им менторство или дополнительные консультации.  

Этот подход позволит университету лучше адаптировать образовательный процесс под запросы разных категорий обучающихся.